This notebook contains ex, that teaches how to use HF datasets. This is an important skill for training purposes. We will be taking guide from ChatGPT for taking suggessions for training a BERT model on small dataset, which takes less resources.

My quarry:
`I want to use a small dataset to pretrain BERT, to do Q & A using huggingface datasets. Can you give me an example that will run small amout of CPU?`

Our Goal: to use finetuning with a basic model BERT 

This script uses only a small portion of the SQuAD training data (e.g. 100 examples) for demonstration purposes. In real-world use, you would use the full dataset (or another dataset) and adjust the training parameters accordingly.

In [1]:
%pip install -q transformers python-dotenv torch datasets transformers[torch] evaluate
#%pip install --upgrade jupyter ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
from dotenv import load_dotenv
import torch
import transformers
import datasets
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
)

load_dotenv()


/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
print("Transformers version:", transformers.__version__)
print("Datasets version:", datasets.__version__)

Transformers version: 4.48.3
Datasets version: 3.3.0


In [4]:

# 1. Load a small subset of the SQuAD dataset
dataset = load_dataset("squad")
# For demonstration, use only the first 10 training examples
small_train_dataset = dataset["train"].select(range(10))
small_eval_dataset = dataset["validation"].select(range(10))



Generating validation split: 100%|██████████| 10570/10570 [00:00<00:00, 529687.61 examples/s]


In [5]:
# 2. Load the tokenizer and model
model_checkpoint = "bert-base-uncased"  # a small BERT model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)



Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# 3. Preprocessing the data
max_length = 128  # reduced max length for the context + question, to lower memory usage
doc_stride = 64  # reduced stride for splitting up long documents

def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")
    
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []
    
    for i, offsets in enumerate(offset_mapping):
        # The corresponding example index
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set start and end positions to the CLS index.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(0)
            tokenized_examples["end_positions"].append(0)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])
            
            # Find the start and end token indices that span the answer.
            token_start_index = 0
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= 0:
                token_start_index += 1

            token_end_index = len(offsets) - 1
            while token_end_index >= 0 and offsets[token_end_index][1] >= max_length:
                token_end_index -= 1

            # If the answer is not fully inside the context, label it (0, 0)
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(0)
                tokenized_examples["end_positions"].append(0)
            else:
                # Otherwise find the token indices corresponding to the answer
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_position = token_start_index - 1

                while token_end_index >= 0 and offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_position = token_end_index + 1

                tokenized_examples["start_positions"].append(start_position)
                tokenized_examples["end_positions"].append(end_position)
    
    return tokenized_examples

# Preprocess the train and evaluation datasets
train_dataset = small_train_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=small_train_dataset.column_names,
)
eval_dataset = small_eval_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=small_eval_dataset.column_names,
)



Map: 100%|██████████| 10/10 [00:00<00:00, 798.09 examples/s]


In [7]:
# 4. Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=10,                   # more frequent evaluation on 10 steps (batches) for demonstration
    learning_rate=2e-5,
    per_device_train_batch_size=1,  # small batch size for CPU
    per_device_eval_batch_size=1,
    num_train_epochs=1,             # only 1 epoch for demonstration
    weight_decay=0.01,
    logging_steps=5,                # more frequent logging on 5 steps (batches) for demonstration
    save_steps=10,
    disable_tqdm=False,
)



/home/codespace/.python/current/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:

# Import the load_metric function and define the metric
from evaluate import load as load_metric

# 5. Define a metric (using squad metric for example)
squad_metric = load_metric("squad")

def compute_metrics(p):
    predictions, labels = p
    # In a real scenario, you would post-process the predictions to convert token indices to text
    # For this small demo, we simply return an empty dictionary.
    return {}



In [9]:
# 6. Create the Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=default_data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)



In [10]:
# 7. Start training
trainer.train()

Step,Training Loss,Validation Loss
10,4.093300,4.184247
20,3.331100,3.834701
30,2.778100,3.667915


TrainOutput(global_step=33, training_loss=3.4859717108986596, metrics={'train_runtime': 97.6499, 'train_samples_per_second': 0.338, 'train_steps_per_second': 0.338, 'total_flos': 2155698243072.0, 'train_loss': 3.4859717108986596, 'epoch': 1.0})

In [11]:

# 8. Evaluate the model (if desired)
eval_results = trainer.evaluate()
print("Evaluation results:", eval_results)


Evaluation results: {'eval_loss': 3.65807843208313, 'eval_runtime': 4.0287, 'eval_samples_per_second': 4.964, 'eval_steps_per_second': 4.964, 'epoch': 1.0}


In [ ]:
# 9. Save the model
trainer.save_model("./my_finetuned_qa_model")        # save the model to disk



: 

# Load Your Model and Tokenizer for Inference

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForQuestionAnswering

# Define the path where your fine-tuned model is saved
model_path = "./my_finetuned_qa_model"

# Load the tokenizer and model from that directory
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)

# Create a question answering pipeline
qa_pipeline = pipeline("question-answering", model=model, tokenizer=tokenizer)


In [ ]:
# Example context and question
context = (
    "The fine-tuned BERT model was trained on a small subset of the SQuAD dataset. "
    "It is now capable of answering questions based on the provided context."
)
question = "What was the model trained on?"

# Use the QA pipeline to get an answer
result = qa_pipeline(question=question, context=context)

print("Answer:", result["answer"])
print("Score:", result["score"])
print("Start index:", result["start"])